In [ ]:
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline
import pandas as pd
import seaborn as sns

from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

from matplotlib import colors
import matplotlib as mpl
from scipy import stats
from sklearn.linear_model import LinearRegression

from pathlib import Path
from SyringePumpAux import *


In [ ]:
#Path settings
figureTargetFolder = Path(r"C:\Users\bfpcr\Documents\device.pump\Exp_Data&Code\AnalysisCode\Figures")
dataFolder = Path(r"C:\Users\bfpcr\Documents\device.pump\Exp_Data&Code\ExperimentalData\FlowData")

#Global Viz settings
sns.set_style('darkgrid') # darkgrid, white grid, dark, white and ticks
plt.rc('axes', titlesize=18)     # fontsize of the axes title
plt.rc('axes', labelsize=14)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=13)    # fontsize of the tick labels
plt.rc('ytick', labelsize=13)    # fontsize of the tick labels
plt.rc('legend', fontsize=13)    # legend fontsize
plt.rc('font', size=13)          # controls default text sizes

mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rcParams['font.family'] = 'Arial'

default_fig_size = (6,4)
default_marker_size = 10.

In [ ]:
#HardCoded Variables
SyringeRadiusInfo = {"Hamilton_5ml": 5.15,
                    "Hamilton_10ml": 7.3} #millimeters

StepSizeMultiplier = {"1x":1,
                     "1x2":0.5,
                     "1x4":0.25,
                     "1x8":0.125,
                     "1x16":0.0625} 
                     
MotorFullStepRotation = 1.8 #degrees
ScrewPitch = 0.8 #millimeters per 360 degrees
fullStepStroke =  (MotorFullStepRotation/360.0) * ScrewPitch # millimeters
fullStepVolume_lambdaFNC = lambda r : (np.pi * (r ** 2) ) * fullStepStroke #cubic millimeters (= 1 microliter)


color_dict = {"Fisher_10ml": '#FF3333',
              "Hamilton_5ml": '#00EA00',
              "terumo_6ml": '#0450FF'} #millimeters


finalVolumeRatio = 0.9

In [ ]:
# Construct df with all metadatat")

AvailableFolders = [x for x in dataFolder.iterdir() if x.is_dir()]


SyringeData = pd.DataFrame(columns = 
                           ['fullDir', 
                            'fName', 
                            'PumpID', 
                            'SyringeID', 
                            'NSteps', 
                            'StepSize', 
                            'Date'])

for iDir in range(0, len(AvailableFolders)):
    CSVList = [f for f in AvailableFolders[iDir].glob('*.csv')]
    for ii, f in enumerate(CSVList):
        CSVList 
        parsedDict = parseFileName(f) #expects a filename in the format: "Pump_4_Hamilton_5ml_70steps_1x2Step2021-06-23T17_55_46.csv"
        SyringeData = SyringeData.append(parsedDict, ignore_index=True)

# Add xsection info
SyringeData["SyringeRadius"] = SyringeData["SyringeID"].map(SyringeRadiusInfo)
SyringeData["StepSizeMultiplier"] = SyringeData["StepSize"].map(StepSizeMultiplier)
SyringeData["VolumePerStep"] = SyringeData["StepSizeMultiplier"] * fullStepVolume_lambdaFNC(SyringeData["SyringeRadius"])
SyringeData["VolumePerWaveform"] = SyringeData["VolumePerStep"] * SyringeData["NSteps"]

# Load the actual data
#Convert the single run dictionary to pandas DF and add it to the original dataFrame.
lambFnc = lambda row : pd.DataFrame(list(
    LoadAndExtractData(row["fullDir"],
    finalVolumeWindow = np.array([7, 10]),
    ratioFinalVolume = finalVolumeRatio, stepWin = np.array([-1,15]),
    ).items())).transpose()

bah = SyringeData.apply(lambFnc, axis = 1)

for ii in range(bah.shape[0]):
    bah[ii].columns = bah[ii].iloc[0]
    bah[ii] = bah[ii].drop(0)
#SyringeData["dataDict"][0]["rawData"].head(5)
temp_df = bah[0]
for ii in range(1,bah.shape[0]):
    temp_df = pd.concat([temp_df, bah[ii]])
temp_df = temp_df.set_index(SyringeData.index)
SyringeData = SyringeData.join(temp_df, how = 'inner')
SyringeData.head(2)

In [ ]:
#Add the mean delivered volume
SyringeData["MeanDeliveredVolume"] = SyringeData.apply(lambda row : np.nanmean(row["finalVolume"]), axis = 1)


In [ ]:
# Averages of averages

colors = cm.get_cmap('viridis', len(SyringeData["NSteps"].unique()))

nstepsLim = 8

fig= plt.figure(figsize= default_fig_size)
ax = fig.add_subplot(1, 1, 1)
iVol = 0

filt_SyringeData = SyringeData[SyringeData["NSteps"]> nstepsLim]
for name, group in filt_SyringeData.groupby(["NSteps"]):
    iVol+=1
    trialSize = np.stack(group.head(1)["volumeAligned"]).shape[2]
    nTrials = group.apply(lambda group : group["volumeAligned"].shape[0], axis = 1)
    maxTrials = np.max(nTrials)
    AllTrials = np.reshape(np.stack(group["volumeAligned"].to_numpy(), axis = 0), [-1, trialSize])
    for jj, row in group.iterrows():
        ts_aligned = row["ts_aligned"]
        for ii, trialData in enumerate(row["volumeAligned"]):
            ax.plot(ts_aligned,trialData, lw = 1.5, color = colors(iVol), alpha = 0.5)

    meanDeliveryTrace = np.mean(AllTrials, axis=0)
    ax.plot(ts_aligned,meanDeliveryTrace, lw = 3, color = colors(iVol), label = name)



ax.set_xlim([-1, 10])
ax.set_ylim([-0.5, 35])
ax.set_ylabel(r'$\Delta$ Volume (capillary x-section)')
ax.set_xlabel('Time since pulse (s)')
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.set_title("flow rate, raw data")
saveFigure(fig, rootSaveFolder = figureTargetFolder)
plt.show()


v_end = 25

fig= plt.figure(figsize= default_fig_size)
ax = fig.add_subplot(1, 1, 1)

iVol = 0
filt_SyringeData = SyringeData[SyringeData["NSteps"]> nstepsLim]
for name, group in filt_SyringeData.groupby(["NSteps"]):
    iVol += 1
    trialSize = np.stack(group.head(1)["volumeAligned"]).shape[2]
    nTrials = group.apply(lambda group : group["volumeAligned"].shape[0], axis = 1)
    maxTrials = np.max(nTrials)
    AllTrials = np.reshape(np.stack(group["volumeAligned"].to_numpy(), axis = 0), [-1, trialSize])
    ll = []
    for jj, row in group.iterrows():
        ts_aligned = row["ts_aligned"]
        for ii, trialData in enumerate(row["volumeAligned"]):
            idx_cap = np.argmin(np.abs(trialData - v_end))
            idx_0 = np.where(ts_aligned == 0)[0][0]
            xx = ts_aligned[idx_0:idx_cap].reshape(-1,1)
            yy = trialData[idx_0:idx_cap]
            lm = LinearRegression().fit(xx, yy)
            slope = lm.coef_[0]
            ll.append(slope)

    ll = np.array(ll)
    plt.errorbar(group["NSteps"].values[0], 
        np.mean(ll), yerr = np.std(ll), 
        color = colors(iVol), 
        fmt = 'o', capsize = 0, elinewidth=3, markersize = default_marker_size, label = name)

    #plt.scatter(group["NSteps"].values[0], np.mean(ll), s = default_marker_size**2,color = colors(iVol), label = name)

ax.set_ylabel(r'Estimated flow rate (a.u. $s^{-1})$')
ax.set_xlabel('Interval between pulses (ms??)')
ax.set_yscale('log')
ax.set_ylim(1,100)
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.set_title("flow rate, means")
saveFigure(fig, rootSaveFolder = figureTargetFolder)
plt.show()

In [ ]:
filt_SyringeData = SyringeData[SyringeData["NSteps"]> 8]

fig = plt.figure(figsize=default_fig_size)
axs = fig.add_subplot(1, 1, 1)
for name, group in filt_SyringeData.groupby(["NSteps"]):
    yy = group["timeFinalVolume"].values[0]
    xx = yy*0 + group["NSteps"].values
    #plt.scatter(xx, yy, s=20, facecolors='none')
    plt.plot(xx, yy, 'o', mfc='none')

    plt.scatter(xx[0], np.mean(yy), s = 50 )


#axs.set_xlim([-1, 12])
#axs.set_ylim([-0.5, 32.5])
axs.set_ylabel('Time to deliver ' + str(group["ratioFinalVolume"].values[0] * 100) + '%')
axs.set_xlabel('Interval between pulses (ms??)')
axs.set_title("This does not work since some of the times take longer than the window")
axs.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()
